In [1]:
import nipype
import os,glob,sys,shutil
import nipype.interfaces.fsl as fsl
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
from IPython.display import Image

from nipype import config
# cfg = dict(execution={'remove_unnecessary_outputs': False,
#                      'keep_inputs': True},
#           monitoring={'enabled': True,
#                       'sample_frequency': 5})

In [2]:
subjRootDir = "/data/HCP_BedpostData/"
FULL_SUBJECT_LIST = [x for x in os.listdir(subjRootDir) if os.path.isdir( subjRootDir+'/addlInfo/subjContainer/'+x)]
print(len(FULL_SUBJECT_LIST),"Subjects are potentially available to be processed!")

(711, 'Subjects are potentially available to be processed!')


In [3]:
"""
Setup for Probtrackx2 Computational Pipeline
"""
subj_infosource = pe.Node(interface=util.IdentityInterface(fields=['subject_id']),  name="subj_infosource")
#infosource.iterables = ('subject_id', SampleSubjList)
subj_infosource.iterables = ('subject_id', FULL_SUBJECT_LIST)
### Above just converts the list of subjects into an iterable list I can connect to the next part of the pipeline

In [4]:
# """
# Setup for DataGrabber inputs needed for probtrackx2
# """
datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
        outfields=['nodif_brain_mask','xfm','invxfm','thsamples','phsamples','fsamples','mniROIs']),
        name='datasource')
# create a node to obtain the functional images
datasource.inputs.base_directory = "/data/HCP_BedpostData/"
datasource.inputs.template ='*'
datasource.inputs.sort_filelist = True
datasource.inputs.field_template = dict(
    thsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
    fsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
    phsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
    nodif_brain_mask='%s/T1w/Diffusion.bedpostX/%s.nii*', 
    xfm='%s/MNINonLinear/xfms/%s.nii*',
    invxfm='%s/MNINonLinear/xfms/%s.nii*',
    mniROIs='addlInfo/subjContainer/%s/DTI_ROIs/Human_*_wimt.nii.gz'
    )

datasource.inputs.template_args = dict(
             thsamples = [['subject_id','th1samples']],
             phsamples =  [['subject_id','ph1samples']],
             fsamples =  [['subject_id','f1samples']],
             nodif_brain_mask = [['subject_id','nodif_brain_mask']],
             xfm = [['subject_id','acpc_dc2standard']],
             invxfm = [['subject_id', 'standard2acpc_dc']],
            mniROIs=[['subject_id']]    
        )

In [5]:
pbx2 = pe.MapNode(interface=fsl.ProbTrackX2(), name='pbx2', iterfield=['seed'])
pbx2.inputs.c_thresh = 0.2   # -c 0.2   F cutoff
pbx2.inputs.n_steps = 2000   # -S 2000
pbx2.inputs.step_length = 0.5 # --steplength=0.5
pbx2.inputs.n_samples = 25000  # -P 5000
pbx2.inputs.opd = True
pbx2.inputs.loop_check = True
pbx2.inputs.correct_path_distribution = True # -pd  i.e. distance correction

In [ ]:
#runpbx.connect(subj_infosource,'subject_id',datasource,'subject_id')
runpbx2  = pe.Workflow(name="runpbx2_gpu_dtispace")
runpbx2.base_dir = "/data/NipypeScratch/"

samples_base_name_fxn = lambda x : x.replace('_th1samples.nii.gz','')

runpbx2.connect(subj_infosource,'subject_id',datasource,'subject_id')

# ### Connect the dti_datasource to the pbx2 command
runpbx2.connect( datasource,'thsamples',pbx2,'thsamples')
runpbx2.connect( datasource,'phsamples',pbx2,'phsamples')
runpbx2.connect( datasource,'fsamples',pbx2,'fsamples')
runpbx2.connect( datasource,'nodif_brain_mask',pbx2,'mask')
runpbx2.connect( datasource, ('thsamples', samples_base_name_fxn ), pbx2,'samples_base_name') ###  NOTE THIS IS A WEIRD TUPLE IS


runpbx2.connect( datasource,'mniROIs', pbx2,'seed')  #pbx2 is a mapnode, so it will run each ROI separately
#runpbx2.run(plugin='MultiProc', plugin_args={'n_procs' : 1})
runpbx2.run()

180912-17:07:15,623 nipype.workflow INFO:
	 Workflow runpbx2_gpu_dtispace settings: ['check', 'execution', 'logging', 'monitoring']
180912-17:07:20,246 nipype.workflow INFO:
	 Running serially.
180912-17:07:20,273 nipype.workflow INFO:
	 [Node] Setting-up "runpbx2_gpu_dtispace.datasource" in "/data/NipypeScratch/runpbx2_gpu_dtispace/_subject_id_132118/datasource".
180912-17:07:20,296 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
180912-17:07:20,325 nipype.workflow INFO:
	 [Node] Finished "runpbx2_gpu_dtispace.datasource".
180912-17:07:20,329 nipype.workflow INFO:
	 [Node] Setting-up "runpbx2_gpu_dtispace.pbx2" in "/data/NipypeScratch/runpbx2_gpu_dtispace/_subject_id_132118/pbx2".
180912-17:07:20,383 nipype.workflow INFO:
	 [Node] Setting-up "_pbx20" in "/data/NipypeScratch/runpbx2_gpu_dtispace/_subject_id_132118/pbx2/mapflow/_pbx20".
180912-17:07:20,406 nipype.workflow INFO:
	 [Node] Running "_pbx20" ("nipype.interfaces.fsl.dti.ProbTrackX2"), 

180912-17:08:29,518 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.421278:Iteration 50 out of 105
180912-17:08:29,520 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.421278:Iteration 51 out of 105
180912-17:08:29,521 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.421278:Iteration 52 out of 105
180912-17:08:29,523 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.421278:Iteration 53 out of 105
180912-17:08:29,525 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.421278:Iteration 54 out of 105
180912-17:08:29,526 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.421278:Iteration 55 out of 105
180912-17:08:29,527 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.421278:Iteration 56 out of 105
180912-17:08:29,529 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.421278:Iteration 57 out of 105
180912-17:08:29,532 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.421278:Iteration 58 out of 105
180912-17:08:29,533 nipype.interface INFO:
	 stdout 2018-09-12T17:08:29.4

180912-17:09:26,277 nipype.interface INFO:
	 stdout 2018-09-12T17:09:26.271984:Free memory after copying masks: 10786504704 ---- Total memory: 12652838912
180912-17:09:26,278 nipype.interface INFO:
	 stdout 2018-09-12T17:09:26.271984:Running 447522 streamlines in parallel using 2 STREAMS
180912-17:09:26,280 nipype.interface INFO:
	 stdout 2018-09-12T17:09:26.271984:Total number of streamlines: 11150000
180912-17:09:26,281 nipype.interface INFO:
	 stdout 2018-09-12T17:09:26.271984:Free memory before running iterations: 1267531776 ---- Total memory: 12652838912
180912-17:09:26,283 nipype.interface INFO:
	 stdout 2018-09-12T17:09:26.271984:Iteration 1 out of 50
180912-17:09:26,285 nipype.interface INFO:
	 stdout 2018-09-12T17:09:26.271984:Iteration 2 out of 50
180912-17:09:26,287 nipype.interface INFO:
	 stdout 2018-09-12T17:09:26.271984:Iteration 3 out of 50
180912-17:09:26,289 nipype.interface INFO:
	 stdout 2018-09-12T17:09:26.271984:Iteration 4 out of 50
180912-17:09:26,291 nipype.int

180912-17:10:21,674 nipype.interface INFO:
	 stdout 2018-09-12T17:10:21.664930:Free memory after copying masks: 10786504704 ---- Total memory: 12652838912
180912-17:10:21,677 nipype.interface INFO:
	 stdout 2018-09-12T17:10:21.664930:Running 447522 streamlines in parallel using 2 STREAMS
180912-17:10:21,679 nipype.interface INFO:
	 stdout 2018-09-12T17:10:21.664930:Total number of streamlines: 12275000
180912-17:10:21,680 nipype.interface INFO:
	 stdout 2018-09-12T17:10:21.664930:Free memory before running iterations: 1267531776 ---- Total memory: 12652838912
180912-17:10:21,682 nipype.interface INFO:
	 stdout 2018-09-12T17:10:21.664930:Iteration 1 out of 55
180912-17:10:21,683 nipype.interface INFO:
	 stdout 2018-09-12T17:10:21.664930:Iteration 2 out of 55
180912-17:10:21,685 nipype.interface INFO:
	 stdout 2018-09-12T17:10:21.664930:Iteration 3 out of 55
180912-17:10:21,686 nipype.interface INFO:
	 stdout 2018-09-12T17:10:21.664930:Iteration 4 out of 55
180912-17:10:21,688 nipype.int

180912-17:11:01,222 nipype.interface INFO:
	 stdout 2018-09-12T17:11:01.217091:
180912-17:11:17,877 nipype.interface INFO:
	 stdout 2018-09-12T17:11:17.877202:
180912-17:11:17,879 nipype.interface INFO:
	 stdout 2018-09-12T17:11:17.877202:...................Allocated GPU 0...................
180912-17:11:17,880 nipype.interface INFO:
	 stdout 2018-09-12T17:11:17.877202:Free memory at the beginning: 12197888000 ---- Total memory: 12652838912
180912-17:11:17,882 nipype.interface INFO:
	 stdout 2018-09-12T17:11:17.877202:Free memory after copying masks: 10786504704 ---- Total memory: 12652838912
180912-17:11:17,884 nipype.interface INFO:
	 stdout 2018-09-12T17:11:17.877202:Running 447522 streamlines in parallel using 2 STREAMS
180912-17:11:17,886 nipype.interface INFO:
	 stdout 2018-09-12T17:11:17.877202:Total number of streamlines: 9175000
180912-17:11:17,888 nipype.interface INFO:
	 stdout 2018-09-12T17:11:17.877202:Free memory before running iterations: 1267531776 ---- Total memory: 12

...

180912-17:47:50,710 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Iteration 16 out of 25
180912-17:47:50,712 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Iteration 17 out of 25
180912-17:47:50,713 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Iteration 18 out of 25
180912-17:47:50,715 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Iteration 19 out of 25
180912-17:47:50,716 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Iteration 20 out of 25
180912-17:47:50,718 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Iteration 21 out of 25
180912-17:47:50,719 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Iteration 22 out of 25
180912-17:47:50,721 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Iteration 23 out of 25
180912-17:47:50,723 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Iteration 24 out of 25
180912-17:47:50,725 nipype.interface INFO:
	 stdout 2018-09-12T17:47:50.677810:Ite

180912-17:48:54,21 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iteration 35 out of 100
180912-17:48:54,23 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iteration 36 out of 100
180912-17:48:54,24 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iteration 37 out of 100
180912-17:48:54,26 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iteration 38 out of 100
180912-17:48:54,27 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iteration 39 out of 100
180912-17:48:54,29 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iteration 40 out of 100
180912-17:48:54,30 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iteration 41 out of 100
180912-17:48:54,32 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iteration 42 out of 100
180912-17:48:54,33 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iteration 43 out of 100
180912-17:48:54,35 nipype.interface INFO:
	 stdout 2018-09-12T17:48:53.961854:Iter

180912-17:48:55,155 nipype.interface INFO:
	 stdout 2018-09-12T17:48:55.155226:PROBTRACKX2 VERSION GPU
180912-17:48:55,157 nipype.interface INFO:
	 stdout 2018-09-12T17:48:55.155226:Log directory is: /data/NipypeScratch/runpbx2_gpu_dtispace/_subject_id_835657/pbx2/mapflow/_pbx21
180912-17:48:55,159 nipype.interface INFO:
	 stdout 2018-09-12T17:48:55.155226:Running in seedmask mode
180912-17:49:32,126 nipype.interface INFO:
	 stdout 2018-09-12T17:49:32.126399:Number of Seeds: 458
180912-17:49:32,128 nipype.interface INFO:
	 stdout 2018-09-12T17:49:32.126399:
180912-17:49:32,130 nipype.interface INFO:
	 stdout 2018-09-12T17:49:32.130395:Time Loading Data: 37 seconds
180912-17:49:32,131 nipype.interface INFO:
	 stdout 2018-09-12T17:49:32.130395:
180912-17:49:45,310 nipype.interface INFO:
	 stdout 2018-09-12T17:49:45.310312:
180912-17:49:45,312 nipype.interface INFO:
	 stdout 2018-09-12T17:49:45.310312:...................Allocated GPU 0...................
180912-17:49:45,313 nipype.interfa

180912-17:49:46,517 nipype.interface INFO:
	 stdout 2018-09-12T17:49:46.517345:PROBTRACKX2 VERSION GPU
180912-17:49:46,520 nipype.interface INFO:
	 stdout 2018-09-12T17:49:46.517345:Log directory is: /data/NipypeScratch/runpbx2_gpu_dtispace/_subject_id_835657/pbx2/mapflow/_pbx22
180912-17:49:46,522 nipype.interface INFO:
	 stdout 2018-09-12T17:49:46.517345:Running in seedmask mode
180912-17:50:23,578 nipype.interface INFO:
	 stdout 2018-09-12T17:50:23.577900:Number of Seeds: 445
180912-17:50:23,582 nipype.interface INFO:
	 stdout 2018-09-12T17:50:23.577900:
180912-17:50:23,584 nipype.interface INFO:
	 stdout 2018-09-12T17:50:23.577900:Time Loading Data: 37 seconds
180912-17:50:23,586 nipype.interface INFO:
	 stdout 2018-09-12T17:50:23.577900:
180912-17:50:37,701 nipype.interface INFO:
	 stdout 2018-09-12T17:50:37.701311:
180912-17:50:37,703 nipype.interface INFO:
	 stdout 2018-09-12T17:50:37.701311:...................Allocated GPU 0...................
180912-17:50:37,704 nipype.interfa

180912-17:50:38,913 nipype.interface INFO:
	 stdout 2018-09-12T17:50:38.913503:PROBTRACKX2 VERSION GPU
180912-17:50:38,917 nipype.interface INFO:
	 stdout 2018-09-12T17:50:38.913503:Log directory is: /data/NipypeScratch/runpbx2_gpu_dtispace/_subject_id_835657/pbx2/mapflow/_pbx23
180912-17:50:38,919 nipype.interface INFO:
	 stdout 2018-09-12T17:50:38.913503:Running in seedmask mode


In [ ]:
graphFile = runpbx2.write_graph(graph2use='orig',simple_form=False)
#runpbx2.write_graph(graph2use='colored',simple_form=True)
## options are flat , exec

# [u'orig', u'flat', u'hierarchical', u'exec', u'colored']


#Image('/data/NipypeScratch/runpbx2_gpu_dtispace/graph_detailed.png')
Image(graphFile)

In [ ]:
### Each probtrackx command for the HCP data set was using about 2% of the memory.. it varied from 1.4 --> 2.3

### So on this machine, we could probably specify 36 threads... even 40 although that may start getting dicey



In [ ]:
### Write a data validator that makes sure everything in the datasource actually exists...




In [ ]:
# pbx2.iterables = ['seed']
#pbx2.iterables = ("seed", "datasource.mniROIs")
#pbx2.inputs.lrtarget3= targetMask  ## This is a hack for now
#pbx2.inputs.seed = seedMask
#pbx2.inputs.omatrix3 = True   # --omatrix3
#pbx2.inputs.target3 = targetMask # --target3=$WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_GM.nii.gz "; 
# pbx2.inputs.onewaycondition = True # --onewaycondition


[Mandatory]
fsamples: (a list of items which are an existing file name)
mask: (an existing file name)
        bet binary mask file in diffusion space
        flag: -m %s
phsamples: (a list of items which are an existing file name)
seed: (an existing file name or a list of items which are an existing
         file name or a list of items which are a list of from 3 to 3 items
         which are an integer (int or long))
        seed volume(s), or voxel(s)or freesurfer label file
        flag: --seed=%s
thsamples: (a list of items which are an existing file name)

[Optional]
args: (a string)
        Additional parameters to the command
        flag: %s
avoid_mp: (an existing file name)
        reject pathways passing through locations given by this mask
        flag: --avoid=%s
c_thresh: (a float)
        curvature threshold - default=0.2
        flag: --cthr=%.3f
colmask4: (an existing file name)
        Mask for columns of matrix4 (default=seed mask)
        flag: --colmask4=%s
correct_path_distribution: (a boolean)
        correct path distribution for the length of the pathways
        flag: --pd
dist_thresh: (a float)
        discards samples shorter than this threshold (in mm - default=0)
        flag: --distthresh=%.3f
distthresh1: (a float)
        Discards samples (in matrix1) shorter than this threshold (in mm -
        default=0)
        flag: --distthresh1=%.3f
distthresh3: (a float)
        Discards samples (in matrix3) shorter than this threshold (in mm -
        default=0)
        flag: --distthresh3=%.3f
environ: (a dictionary with keys which are a value of type 'str' and
         with values which are a value of type 'str', nipype default value:
         {})
        Environment variables
fibst: (an integer (int or long))
        force a starting fibre for tracking - default=1, i.e. first fibre
        orientation. Only works if randfib==0
        flag: --fibst=%d
fopd: (an existing file name)
        Other mask for binning tract distribution
        flag: --fopd=%s
force_dir: (a boolean, nipype default value: True)
        use the actual directory name given - i.e. do not add + to make a
        new directory
        flag: --forcedir
ignore_exception: (a boolean, nipype default value: False)
        Print an error message instead of throwing an exception in case the
        interface fails to run
inv_xfm: (a file name)
        transformation matrix taking DTI space to seed space (compulsory
        when using a warp_field for seeds_to_dti)
        flag: --invxfm=%s
loop_check: (a boolean)
        perform loop_checks on paths - slower, but allows lower curvature
        threshold
        flag: --loopcheck
lrtarget3: (an existing file name)
        Column-space mask used for Nxn connectivity matrix
        flag: --lrtarget3=%s
meshspace: ('caret' or 'freesurfer' or 'first' or 'vox')
        Mesh reference space - either "caret" (default) or "freesurfer" or
        "first" or "vox"
        flag: --meshspace=%s
mod_euler: (a boolean)
        use modified euler streamlining
        flag: --modeuler
n_samples: (an integer (int or long), nipype default value: 5000)
        number of samples - default=5000
        flag: --nsamples=%d
n_steps: (an integer (int or long))
        number of steps per sample - default=2000
        flag: --nsteps=%d
network: (a boolean)
        activate network mode - only keep paths going through at least one
        seed mask (required if multiple seed masks)
        flag: --network
omatrix1: (a boolean)
        Output matrix1 - SeedToSeed Connectivity
        flag: --omatrix1
omatrix2: (a boolean)
        Output matrix2 - SeedToLowResMask
        flag: --omatrix2
        requires: target2
omatrix3: (a boolean)
        Output matrix3 (NxN connectivity matrix)
        flag: --omatrix3
        requires: target3, lrtarget3
omatrix4: (a boolean)
        Output matrix4 - DtiMaskToSeed (special Oxford Sparse Format)
        flag: --omatrix4
onewaycondition: (a boolean)
        Apply waypoint conditions to each half tract separately
        flag: --onewaycondition
opd: (a boolean, nipype default value: True)
        outputs path distributions
        flag: --opd
os2t: (a boolean)
        Outputs seeds to targets
        flag: --os2t
out_dir: (an existing directory name)
        directory to put the final volumes in
        flag: --dir=%s
output_type: ('NIFTI_PAIR' or 'NIFTI_PAIR_GZ' or 'NIFTI_GZ' or
         'NIFTI')
        FSL output type
rand_fib: (0 or 1 or 2 or 3)
        options: 0 - default, 1 - to randomly sample initial fibres (with f
        > fibthresh), 2 - to sample in proportion fibres (with f>fibthresh)
        to f, 3 - to sample ALL populations at random (even if f<fibthresh)
        flag: --randfib=%d
random_seed: (a boolean)
        random seed
        flag: --rseed
s2tastext: (a boolean)
        output seed-to-target counts as a text file (useful when seeding
        from a mesh)
        flag: --s2tastext
sample_random_points: (a boolean)
        sample random points within seed voxels
        flag: --sampvox
samples_base_name: (a string, nipype default value: merged)
        the rootname/base_name for samples files
        flag: --samples=%s
seed_ref: (an existing file name)
        reference vol to define seed space in simple mode - diffusion space
        assumed if absent
        flag: --seedref=%s
simple: (a boolean)
        rack from a list of voxels (seed must be a ASCII list of
        coordinates)
        flag: --simple
step_length: (a float)
        step_length in mm - default=0.5
        flag: --steplength=%.3f
stop_mask: (an existing file name)
        stop tracking at locations given by this mask file
        flag: --stop=%s
target2: (an existing file name)
        Low resolution binary brain mask for storing connectivity
        distribution in matrix2 mode
        flag: --target2=%s
target3: (an existing file name)
        Mask used for NxN connectivity matrix (or Nxn if lrtarget3 is set)
        flag: --target3=%s
target4: (an existing file name)
        Brain mask in DTI space
        flag: --target4=%s
target_masks: (a list of items which are a file name)
        list of target masks - required for seeds_to_targets classification
        flag: --targetmasks=%s
terminal_output: ('stream' or 'allatonce' or 'file' or 'none')
        Control terminal output: `stream` - displays to terminal immediately
        (default), `allatonce` - waits till command is finished to display
        output, `file` - writes output to file, `none` - output is ignored
use_anisotropy: (a boolean)
        use anisotropy to constrain tracking
        flag: --usef
verbose: (0 or 1 or 2)
        Verbose level, [0-2].Level 2 is required to output particle files.
        flag: --verbose=%d
waycond: ('OR' or 'AND')
        Waypoint condition. Either "AND" (default) or "OR"
        flag: --waycond=%s
wayorder: (a boolean)
        Reject streamlines that do not hit waypoints in given order. Only
        valid if waycond=AND
        flag: --wayorder
waypoints: (an existing file name)
        waypoint mask or ascii list of waypoint masks - only keep paths
        going through ALL the masks
        flag: --waypoints=%s
xfm: (an existing file name)
        transformation matrix taking seed space to DTI space (either FLIRT
        matrix or FNIRT warp_field) - default is identity
        flag: --xfm=%s

In [ ]:
# roi_infosource = pe.Node(interface=util.IdentityInterface(fields=['mniROIs']),  name="roi_infosource")
# #infosource.iterables = ('subject_id', SampleSubjList)
# #roi_infosource.iterables = ('subject_id', FULL_SUBJECT_LIST)


# roiSplit = pe.Node(interface=util.Split(splits=[1,1,1,1,1,1]),name='roiSplit')


In [ ]:
# runpbx2.connect( dti_datasource, ('thsamples',samples_base_name_fxn) ,pbx2,'samples_base_name')
# # Run tractography with warp to/from template space
# $statement .= " -s $WORKINGDATAPATH/Fox_" . $subj[$k] . "/data.bedpostX/merged "; 
# $statement .= " -x $WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_WM.nii.gz ";  ## check downsample
# $statement .= " -l   --fibthresh=0.1 --randfib=0 ";  ## check # of samples
# $statement .= " --forcedir --opd ";
# $statement .= " --dir=$WORKINGDATAPATH/Fox_" . $subj[$k] . "/data.bedpostX/WholeBrainMatrixConnectivity_TemplateSpace_fullres/ ";
# print "$statement \n";

In [ ]:
# --pd   correct_path_distribution 
#-l
#-c 0.2    --->> c_thresh
#-S 2000   
#--steplength=0.5
#-P 5000    n_samples
#--fibthresh=0.1   
#--randfib=0  rand_fib
#-s /home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/Subj_026/data.bedpostX/merged 
#-m /home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/Subj_026/data/nodif_brain_mask.nii.gz
#-x /home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/ROIs/Human_BasalForebrain_Left.nii.gz 
#--target2=/home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/ROIs/MNI152_T1_1mm_brain_mask.nii.gz 
#--xfm=/home/ehecht/TEMPLETON/Templeton_Registration/xfms/Subj_026_Scan2_MNI_fnirt_struct_6dof-flirt_nodif.nii.gz
#--invxfm=/home/ehecht/TEMPLETON/Templeton_Registration/xfms/Subj_026_Scan2_nodif_6dof-flirt_struct_fnirt_MNI.nii.gz 
#--forcedir 
#--opd
#--dir=/home/ehecht/TEMPLETON/Templeton_DTI_Analysis/Data/Subj_026/BasalForebrain_Left_TemplateSpace/

In [ ]:
#  pbx2.inputs.seed = 'seed_source.nii.gz'
# >>> pbx2.inputs.thsamples = 'merged_th1samples.nii.gz'
# >>> pbx2.inputs.fsamples = 'merged_f1samples.nii.gz'
# >>> pbx2.inputs.phsamples = 'merged_ph1samples.nii.gz'
# >>> pbx2.inputs.mask = 'nodif_brain_mask.nii.gz'
# >>> pbx2.inputs.out_dir = '.'
# >>> pbx2.inputs.n_samples = 3
# >>> pbx2.inputs.n_steps = 10
# >>> pbx2.cmdline

# datasource.inputs.template = '%s/T1w/Diffusion/%s'
# datasource.inputs.template_args = dict(dwi=[['subject_id', 'data.nii.gz']],
#                                        bvecs=[['subject_id', 'bvecs']],
#                                        bvals=[['subject_id', 'bvals']],
#                                        nodif_brain_mask=[['subject_id','nodif_brain_mask.nii.gz']])
## Just mapped each subject to the corresponding bvec,bvals, brain mask and preprocessed DWI data
### Create the Node for DTIFIT
# dtifit = pe.Node(interface=fsl.DTIFit(), name='dtifit')


# gen_fa = pe.Workflow(name="gen_fa")
# gen_fa.base_dir = '/data/HCP_Data/NipypeScratch/'
# gen_fa.connect(subject_id_infosource, 'subject_id', datasource, 'subject_id')

# gen_fa.connect(subject_id_infosource, 'subject_id', dtifit, 'base_name')
# gen_fa.connect(datasource, 'bvecs', dtifit, 'bvecs')
# gen_fa.connect(datasource, 'bvals', dtifit, 'bvals')
# gen_fa.connect(datasource, 'nodif_brain_mask', dtifit, 'mask')
# gen_fa.connect(datasource, 'dwi', dtifit, 'dwi')

In [ ]:
print(gen_fa.write_graph(graph2use='colored',simple_form=False))
Image('/data/HCP_Data/NipypeScratch/gen_fa/graph.png')
#Image('/data/HCP_Data/NipypeScratch/gen_fa/graph.png')

In [ ]:
gen_fa.run(plugin='MultiProc', plugin_args={'n_procs' : 20})


In [ ]:
#!ls /data/HCP_Data/HCP_BedpostData/106016/T1w/Diffusion/

In [ ]:
datasink = pe.Node(interface=nio.DataSink(), name="datasink")
datasink.inputs.base_directory = os.path.join(os.path.abspath(working_data_dir),
                                              'sri_results')
datasink.inputs.parameterization = False
gen_fa.connect(dtifit, 'FA', datasink, 'FA')
gen_fa.connect(dtifit, 'MD', datasink, 'MD')

In [ ]:
# """
# Setup for DataGrabber inputs needed for probtrackx2
# """
datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
        outfields=['nodif_brain_mask','xfm','invxfm','thsamples','phsamples','fsamples','lhyp',
                   'rhyp','hyp','lbf','rbf','bf','mniROIs']),
        name='datasource')
# create a node to obtain the functional images
datasource.inputs.base_directory = "/data/HCP_BedpostData/"
datasource.inputs.template ='*'
datasource.inputs.sort_filelist = True
datasource.inputs.field_template = dict(
    thsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
    fsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
    phsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
    nodif_brain_mask='%s/T1w/Diffusion.bedpostX/%s.nii*', 
    xfm='%s/MNINonLinear/xfms/%s.nii*',
    invxfm='%s/MNINonLinear/xfms/%s.nii*',
    lhyp='addlInfo/subjContainer/%s/DTI_ROIs/Human_Hypothalamus_Left_wimt.nii.gz',
    rhyp='addlInfo/subjContainer/%s/DTI_ROIs/Human_Hypothalamus_Right_wimt.nii.gz',
    hyp='addlInfo/subjContainer/%s/DTI_ROIs/Human_Hypothalamus_Bilat_wimt.nii.gz',
    lbf='addlInfo/subjContainer/%s/DTI_ROIs/Human_BasalForebrain_Left_wimt.nii.gz',
    rbf='addlInfo/subjContainer/%s/DTI_ROIs/Human_BasalForebrain_Right_wimt.nii.gz',
    bf='addlInfo/subjContainer/%s/DTI_ROIs/Human_BasalForebrain_Bilat_wimt.nii.gz',
    mniROIs='addlInfo/subjContainer/%s/DTI_ROIs/Human_*_wimt.nii.gz'
    )

# Human_BasalForebrain_Bilat_wimt.nii.gz  Human_BasalForebrain_Right_wimt.nii.gz  Human_Hypothalamus_Left_wimt.nii.gz
# Human_BasalForebrain_Left_wimt.nii.gz   Human_Hypothalamus_Bilat_wimt.nii.gz    Human_Hypothalamus_Right_wimt.nii.gz

datasource.inputs.template_args = dict(
             thsamples = [['subject_id','th1samples']],
             phsamples =  [['subject_id','ph1samples']],
             fsamples =  [['subject_id','f1samples']],
             nodif_brain_mask = [['subject_id','nodif_brain_mask']],
             xfm = [['subject_id','acpc_dc2standard']],
             invxfm = [['subject_id', 'standard2acpc_dc']],
             bf=[['subject_id']], 
             lbf=[['subject_id']],
             rbf=[['subject_id']],
             lhyp=[['subject_id']],
             rhyp=[['subject_id']],
             hyp=[['subject_id']],
            mniROIs=[['subject_id']]    
        )

In [ ]:
from nipype.interfaces import fsl

subjDir = "/data/HCP_Data/HCP_BedpostData/100206/T1w/Diffusion.bedpostX/"

pbx2 = fsl.ProbTrackX2()
pbx2.inputs.seed = '/data/HCP_Data/EHECHT_ROIS/Human_BasalForebrain_Left.nii.gz'
pbx2.inputs.thsamples = subjDir + 'merged_th1samples.nii.gz'
pbx2.inputs.fsamples =  subjDir + 'merged_f1samples.nii.gz'
pbx2.inputs.phsamples =  subjDir + 'merged_ph1samples.nii.gz'
pbx2.inputs.mask =  subjDir + 'nodif_brain_mask.nii.gz'
pbx2.inputs.out_dir = '.'
pbx2.inputs.n_samples = 3
pbx2.inputs.n_steps = 10
pbx2.cmdline
pbx2.run()
#'probtrackx2 --forcedir -m nodif_brain_mask.nii.gz --nsamples=3 --nstep